In [1]:
import sqlite3
import os
import sys

#dim = 64   #32 o 64 o 128
#dir = 'itwac'

#parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

#sql_path = f'{parent_dir}/{dir}/itwac{dim}.sqlite'
#txt_path = f'{parent_dir}/{dir}/itwac{dim}.txt'
#conllu_dir = 'data/profiling_output/11462'

In [11]:
con = sqlite3.connect(sql_path)

In [12]:
cur = con.cursor()

In [13]:
with open(txt_path, 'w+') as out_file:
    for embedding in cur.execute("SELECT * FROM store"):
        str_embedding = [str(el) for el in embedding[:-1]]
        out_file.write('\t'.join(str_embedding)+'\n')

out_file.close()

In [62]:
embeddings_dim = dim
embeddings_path = f'itwac/itwac{embeddings_dim}.txt'

In [2]:
sys.path.append('..')  # Add parent directory to the Python path
from utils.helpers import *

In [63]:
embeddings = load_word_embeddings(parent_dir+"/"+embeddings_path) 

In [64]:
embeddings['vedersi']

array([ 0.15952599, -0.06988297,  0.00461498, -0.14643039, -0.06968652,
        0.12208004, -0.23405366,  0.029918  ,  0.10900551, -0.0650151 ,
       -0.11520433, -0.02036252,  0.22303598, -0.07288922,  0.11827985,
        0.08765573,  0.32444623,  0.02425564, -0.0860287 ,  0.17139778,
       -0.07151143, -0.12564972, -0.00202048, -0.085145  ,  0.2883054 ,
       -0.01258776,  0.13457723, -0.09618071, -0.06798776,  0.08887868,
        0.05748953,  0.11354444,  0.11662324, -0.12872136, -0.05526053,
        0.0794652 ,  0.02450059,  0.12400361,  0.07817179, -0.04936175,
       -0.0552861 , -0.2573702 , -0.10783048, -0.1763874 , -0.07100514,
        0.18446094, -0.08303817, -0.01772626,  0.09265539,  0.12714238,
        0.16492783,  0.00884029, -0.0278984 ,  0.08680572, -0.00204048,
        0.0403213 , -0.06551646, -0.16743493,  0.12575132, -0.18560314,
       -0.17795198,  0.15221214,  0.11182684, -0.14770217])

In [65]:
import os
all_documents_paths = []
for file_name in os.listdir(conllu_dir):
    file_path = os.path.join(conllu_dir, file_name)
    all_documents_paths.append(file_path)

In [66]:
all_documents = []

for document_path in all_documents_paths:
    document_tokens = get_tokens_from_file(document_path, postg=True)
    all_documents.append(document_tokens)

In [67]:
all_documents

[[{'word': 'Wow', 'pos': 'PROPN'},
  {'word': 'io', 'pos': 'PRON'},
  {'word': 'mi', 'pos': 'PRON'},
  {'word': 'chiamo', 'pos': 'VERB'},
  {'word': 'Giada', 'pos': 'PROPN'}],
 [{'word': 'Non', 'pos': 'ADV'},
  {'word': 'per', 'pos': 'ADP'},
  {'word': 'insultare', 'pos': 'VERB'},
  {'word': 'ma', 'pos': 'CCONJ'},
  {'word': 'quando', 'pos': 'SCONJ'},
  {'word': 'gridava', 'pos': 'VERB'},
  {'word': 'sembrava', 'pos': 'VERB'},
  {'word': 'una', 'pos': 'DET'},
  {'word': 'cornacchia', 'pos': 'NOUN'}],
 [{'word': 'Commuovente', 'pos': 'ADV'},
  {'word': '!!!', 'pos': 'PUNCT'},
  {'word': '!!', 'pos': 'PUNCT'}],
 [{'word': 'Renzi', 'pos': 'ADV'},
  {'word': 'non', 'pos': 'ADV'},
  {'word': 'sei', 'pos': 'AUX'},
  {'word': 'collocabile', 'pos': 'ADJ'},
  {'word': 'sulla', 'pos': '_'},
  {'word': 'scala', 'pos': 'NOUN'},
  {'word': 'degli', 'pos': '_'},
  {'word': 'esseri', 'pos': 'NOUN'},
  {'word': 'viventi', 'pos': 'ADJ'}],
 [{'word': 'Renzi', 'pos': 'ADV'},
  {'word': 'fai', 'pos': 'VER

In [3]:
def compute_embeddings_mean(document_embeddings):
    sum_array = np.sum(document_embeddings, axis=0)
    mean_array = np.divide(sum_array, len(document_embeddings))
    return mean_array

def compute_all_embeddings_mean(document_tokens, embeddings_dim):
    document_embeddings = []
    
    for token in document_tokens:
        word = token['word']
        if word in embeddings:
            document_embeddings.append(embeddings[word])
    
    if len(document_embeddings) == 0:
        mean_document_embeddings = np.zeros(embeddings_dim)
    else:
        mean_document_embeddings = compute_embeddings_mean(document_embeddings)
    return mean_document_embeddings

def compute_filtered_embeddings_mean(document_tokens, embeddings_dim):
    document_embeddings = []
    
    for token in document_tokens:
        word = token['word']
        pos = token['pos']
        if word in embeddings and pos in ['ADJ', 'NOUN', 'VERB', 'PREP']:
            document_embeddings.append(embeddings[word])
    
    if len(document_embeddings) == 0:
        mean_document_embeddings = np.zeros(embeddings_dim)
    else:
        mean_document_embeddings = compute_embeddings_mean(document_embeddings)
    return mean_document_embeddings


def compute_filtered_embeddings_sep_means(document_tokens, embeddings_dim):
    adj_embeddings = []
    noun_embeddings = []
    verb_embeddings = []
    prep_embeddings = []
    
    for token in document_tokens:
        word = token['word']
        pos = token['pos']
        if word in embeddings and pos in ['ADJ']:
            adj_embeddings.append(embeddings[word])
        elif word in embeddings and pos in ['NOUN']:
            noun_embeddings.append(embeddings[word])
        elif word in embeddings and pos in ['VERB']:
            verb_embeddings.append(embeddings[word])
        elif word in embeddings and pos in ['PREP']:
            prep_embeddings.append(embeddings[word])
    
    if len(adj_embeddings) == 0:
        mean_adj_embeddings = np.zeros(embeddings_dim)
    else:
        mean_adj_embeddings = compute_embeddings_mean(adj_embeddings)
        
    if len(noun_embeddings) == 0:
        mean_noun_embeddings = np.zeros(embeddings_dim)
    else:
        mean_noun_embeddings = compute_embeddings_mean(noun_embeddings)
        
    if len(verb_embeddings) == 0:
        mean_verb_embeddings = np.zeros(embeddings_dim)
    else:
        mean_verb_embeddings = compute_embeddings_mean(verb_embeddings)

    if len(prep_embeddings) == 0:
        mean_prep_embeddings = np.zeros(embeddings_dim)
    else:
        mean_prep_embeddings = compute_embeddings_mean(prep_embeddings) 
    
    
    mean_document_embeddings = np.concatenate([mean_adj_embeddings, mean_noun_embeddings, mean_verb_embeddings, mean_prep_embeddings], axis=None)
    return mean_document_embeddings

In [4]:
def extract_features(documents, embeddings_dim):
    dataset_features = []
    for document_tokens in documents:
        document_embeddings = compute_all_embeddings_mean(document_tokens, embeddings_dim)
        #document_embeddings = compute_filtered_embeddings_mean(document_tokens, embeddings_dim)
        #document_embeddings = compute_filtered_embeddings_sep_means(document_tokens, embeddings_dim)
        dataset_features.append(document_embeddings)
    return dataset_features

In [68]:
all_features = extract_features(all_documents)


In [69]:
all_labels = create_label_list(all_documents_paths)


In [5]:
def train_test_split(all_features, all_labels, all_documents_paths):
    train_features, train_labels = [], []
    test_features, test_labels = [], []
    
    for features, label,  document_path in zip(all_features, all_labels, all_documents_paths):
        if 'training' in document_path:
            train_features.append(features)
            train_labels.append(label)
        else:
            test_features.append(features)
            test_labels.append(label)
    return train_features, train_labels, test_features, test_labels


In [70]:
train_features, train_labels, test_features, test_labels = train_test_split(all_features, all_labels, all_documents_paths)
len(train_features), len(train_labels), len(test_features), len(test_labels)

(4395, 4395, 4400, 4400)

In [71]:
len(train_labels)

4395

In [72]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
# X_train = np.stack(train_features, axis=0)
X_train = scaler.fit_transform(train_features)

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

# Define the parameter grid to search through
param_grid = {
    'C': [1,10,0.1, 0.01, 0.001],  # Regularization parameter
    'dual': [True, False]
}

# Initialize SVM with linear kernel
svc = LinearSVC()

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, refit=True)

# Fit the grid search to your data
grid_search.fit(X_train, train_labels)

# Get mean test scores across folds
mean_test_scores = grid_search.cv_results_['mean_test_score']

# Print the best parameters and best score
print("Best parameters found:", grid_search.best_params_)
print("Best score found:", grid_search.best_score_)

c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best parameters found: {'C': 10, 'dual': False}
Best score found: 0.5888509670079636


In [30]:
X_test = scaler.transform(test_features)
# Get the best estimator (model) found by grid search
best_model = grid_search.best_estimator_

# Now, you can use this best_model to make predictions on new data
# For example, if you have new data X_new, you can predict its labels as follows:
predictions = best_model.predict(X_test)

In [31]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix
test_predictions = predictions
print(classification_report(test_labels, test_predictions, zero_division=0)) # output_dict=True

              precision    recall  f1-score   support

           F       0.52      0.75      0.62      2200
           M       0.56      0.32      0.40      2200

    accuracy                           0.53      4400
   macro avg       0.54      0.53      0.51      4400
weighted avg       0.54      0.53      0.51      4400



In [6]:
import os
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

# Configurazioni da esplorare
embedding_dim = [32, 64, 128]  # Dimensioni degli embedding
representations = [False, True]  # False = solo "word", True = "word + pos"

# Directory e path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
dir_name = 'itwac'
conllu_dir = 'data/profiling_output/11462'
results = []

# Parametri della GridSearch
param_grid = {
    'C': [10, 1, 0.1, 0.01, 0.001],
    'dual': [True, False]
}

for dim in embedding_dim:
    embeddings_path = f'{dir_name}/itwac{dim}.txt'
    all_documents_paths = []
    for file_name in os.listdir(conllu_dir):
        file_path = os.path.join(conllu_dir, file_name)
        all_documents_paths.append(file_path)
    embeddings = load_word_embeddings(parent_dir+"/"+embeddings_path)
    
    for postg in representations:
        representation_type = "word + pos" if postg else "word"
        print(f"Processing: Embedding Dim={dim}, Representation={representation_type}")

        
        all_documents = []
        for document_path in all_documents_paths:
            document_tokens = get_tokens_from_file(document_path, postg=postg)
            all_documents.append(document_tokens)
        
        # Estrai feature e label
        all_features = extract_features(all_documents, dim)
        print(len(all_features))
        all_labels = create_label_list(all_documents_paths)
        train_features, train_labels, test_features, test_labels = train_test_split(
            all_features, all_labels, all_documents_paths)
        print(len(train_features))
        # Feature extraction e scaling
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(train_features)

        # GridSearch con SVM lineare
        svc = LinearSVC()
        grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, refit=True)
        grid_search.fit(X_train, train_labels)
        
        # Calcola metriche su train e test
        X_test = scaler.transform(test_features)
        predictions = grid_search.best_estimator_.predict(X_test)
        val_acc = grid_search.best_score_  # Media delle validation accuracy nei fold
        test_acc = np.mean(predictions == test_labels)  # Test accuracy
        
        # Salva il risultato
        results.append({
            "embedding_dim": dim,
            "representation": representation_type,
            "val_acc": val_acc,
            "test_acc": test_acc,
            "best_params": grid_search.best_params_
        })

        print("Validation accuracy", val_acc)

# Trova i modelli migliori per validation accuracy
best_val_model = max(results, key=lambda x: x["val_acc"])
best_test_model = max(results, key=lambda x: x["test_acc"])

# Stampa i dettagli del miglior modello
print("\nBest Model by Validation Accuracy:")
print(best_val_model)

print("\nBest Model by Test Accuracy:")
print(best_test_model)

# Confusion matrix e classification report per il miglior modello di validazione
best_dim = best_val_model["embedding_dim"]
best_rep = best_val_model["representation"] == "word + pos"

print(f"\nClassification Report for Best Validation Model (Embedding Dim={best_dim}, Representation={best_rep})")
all_documents = [get_tokens_from_file(doc_path, postg=best_rep) for doc_path in all_documents_paths]
all_features = extract_features(all_documents)
all_labels = create_label_list(all_documents_paths)
train_features, train_labels, test_features, test_labels = train_test_split(
    all_features, all_labels, all_documents_paths)
vectorizer = DictVectorizer()
X_train = vectorizer.fit_transform(train_features)
scaler = MaxAbsScaler()
X_train = scaler.fit_transform(X_train)
X_test = vectorizer.transform(test_features)
X_test = scaler.transform(X_test)

svc = LinearSVC(**best_val_model["best_params"])
svc.fit(X_train, train_labels)
predictions = svc.predict(X_test)

print(classification_report(test_labels, predictions, zero_division=0))
ConfusionMatrixDisplay.from_predictions(test_labels, predictions, xticks_rotation='vertical', cmap='Blues')
plt.show()


Processing: Embedding Dim=32, Representation=word
8795
4395


c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Validation accuracy 0.5865756541524461
Processing: Embedding Dim=32, Representation=word + pos
8795
4395


c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Validation accuracy 0.5865756541524461
Processing: Embedding Dim=64, Representation=word
8795
4395


...

c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number 

Validation accuracy 0.5961319681456201
Processing: Embedding Dim=128, Representation=word + pos
8795
4395


c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number 

Validation accuracy 0.5961319681456201

Best Model by Validation Accuracy:
{'embedding_dim': 128, 'representation': 'word', 'val_acc': 0.5961319681456201, 'test_acc': 0.5372727272727272, 'best_params': {'C': 0.1, 'dual': True}}

Best Model by Test Accuracy:
{'embedding_dim': 128, 'representation': 'word', 'val_acc': 0.5961319681456201, 'test_acc': 0.5372727272727272, 'best_params': {'C': 0.1, 'dual': True}}

Classification Report for Best Validation Model (Embedding Dim=128, Representation=False)


TypeError: extract_features() missing 1 required positional argument: 'embeddings_dim'

In [35]:
train_features

[array([-0.05698564,  0.00395603,  0.01698117, -0.00045815, -0.00759059,
        -0.16711103, -0.00449716,  0.02978932, -0.00366373, -0.00143729,
         0.03695516,  0.00091665, -0.07400634, -0.00193286, -0.00159305,
         0.04446828, -0.01687855,  0.04428902,  0.06611198, -0.0120648 ,
        -0.00558129,  0.09815954,  0.00608429, -0.11648438, -0.04065889,
         0.19600833,  0.04677407, -0.04132922, -0.09944084, -0.13067213,
        -0.10923235,  0.09846224]),
 array([-0.06079666, -0.01098419,  0.00617873,  0.07821682, -0.11652729,
        -0.06216425, -0.03935083,  0.06931643, -0.01284301,  0.08776171,
        -0.06226566,  0.00468118, -0.26576521, -0.10567843, -0.01952167,
        -0.00271442,  0.06548957,  0.09695172, -0.00145199, -0.01728204,
         0.02512525,  0.24919349, -0.0674348 , -0.04058614,  0.09240107,
         0.20673508,  0.04510129,  0.04233606, -0.07702131, -0.01585263,
        -0.07779491,  0.00179264]),
 array([-0.00410491, -0.03670746,  0.00223504, -0.10